# Dataset creation

In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text), "chars\n-----")
print(text[:100])

1115394 chars
-----
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [4]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

encode = lambda word: [stoi[c] for c in word] 
decode = lambda word: "".join([itos[c] for c in word])

print(encode("Hello there"))
print(decode(encode("Hello there")))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
Hello there


Known tokenizers: sentence piece (google), ticktoken (BPE, openAI)

In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
# batch dimention
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in paralell
block_size = 8 # max context_length
def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # beginning of the context
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:", xb.shape, xb, sep="\n")
print("targets:", yb.shape, yb, "----", sep="\n")
# each row is a training example


# time dimention
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(context, "→", target.item())

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
tensor([24]) → 43
tensor([24, 43]) → 58
tensor([24, 43, 58]) → 5
tensor([24, 43, 58,  5]) → 57
tensor([24, 43, 58,  5, 57]) → 1
tensor([24, 43, 58,  5, 57,  1]) → 46
tensor([24, 43, 58,  5, 57,  1, 46]) → 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) → 39
tensor([44]) → 53
tensor([44, 53]) → 56
tensor([44, 53, 56]) → 1
tensor([44, 53, 56,  1]) → 58
tensor([44, 53, 56,  1, 58]) → 46
tensor([44, 53, 56,  1, 58, 46]) → 39
tensor([44, 53, 56,  1, 58, 46, 39]) → 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) → 1
tensor([52]) → 58
tensor([52, 58]) → 1
tensor([52, 58,  1]) → 58
tensor([52, 58,  1, 58]) → 46

It's multiple examples at once. We want to predict 18 → 47; 18, 47 → 56; ...

We'd like the transformers to predict for little and big contexts, up to blocksize.

# Bigram model

In [8]:
import torch.nn as nn
from torch.nn import functional as F
from math import log
torch.manual_seed(1337)

# baseline – context of 1
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C) – (batch, time, channel)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # pytorch expects C in 2nd dimention
            targets = targets.view(B*T) # or -1
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx) # feed all history to the – not really needed here
            # focus only on the last time step (last elem in time dimention)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item(), -log(1/65))

idx = torch.zeros((1,1), dtype=torch.long) # start with a single newline token
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) # 0th batch, because it's the only one

torch.Size([32, 65])
4.878634929656982 4.174387269895637

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [90]:
# train
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for i in range(10000):
    xb, yb = get_batch("test")
    # forward
    logits, loss = m(xb, yb)
    # backward
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # update
    optimizer.step()

print(loss.item())


2.26753830909729


In [95]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist())) # 0th batch, because it's the only one


ONISPENTheagse y hofiniral at, toffthuchot

IOSust s.

ysed;
LOSENIEBe!
ANAPEn
SEN:
I'r, sthou se o-


# Self-attention

In [9]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

Tokens should commuinicate with previous tokens in the sequence.

In [10]:
# bag of words – just average 

# version 1 – explicit loop
xbow = torch.zeros((B, T, C)) 
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b, t] = torch.mean(xprev, 0) # (C,)

# version 2 – matrix multiplication
wei = torch.tril(torch.ones(T, T)) # lower triangular matrix – weights
wei = wei / wei.sum(dim=1, keepdim=True) # normalize

xbow2 = wei @ x #(B, *T*, T) @ (B, T, C) → (B, *T*, C)

# version 3 – softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T)) # how much of each token are we taking; it can be different in the future
wei = wei.masked_fill(tril==0, float('-inf')) # token from the past cannot communicate; where tril is 0, fill with -inf
wei = F.softmax(wei, dim=-1) # normalize; wei is now the same as in 2nd version

xbow3 = wei @ x


torch.allclose(xbow, xbow3, atol=1e-7)

True

In [11]:
# mean but only for the context
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3))) # lower triangular matrix
a /= torch.sum(a, dim=1, keepdim=True) # normalize
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(f"{a=}")
print(f"{b=}")
print(f"{c=}")


a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Every single token has query (what an I looking for) and a key (what do I contain). Later we do q * k for key from all tokens to get the attention score.

E.g. k: I'm a consonant on the posision 8 

q: i'm looking for a vowel on the positions 1-4

In [12]:
# version 4 – self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channel
x = torch.randn(B, T, C)

# single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, H=16)
q = query(x) # (B, T, H=16)
wei = q @ k.transpose(-2, -1) # (B, T, H) @ (B, H, T) → (B, T, T)
# wei = torch.zeros((T, T)) # not 0 anymore

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [13]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5599, 0.4401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3220, 0.2016, 0.4764, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1640, 0.0815, 0.2961, 0.4585, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2051, 0.3007, 0.1894, 0.1808, 0.1241, 0.0000, 0.0000, 0.0000],
        [0.0600, 0.1273, 0.0291, 0.0169, 0.0552, 0.7114, 0.0000, 0.0000],
        [0.1408, 0.1025, 0.1744, 0.2038, 0.1690, 0.0669, 0.1426, 0.0000],
        [0.0223, 0.1086, 0.0082, 0.0040, 0.0080, 0.7257, 0.0216, 0.1016]],
       grad_fn=<SelectBackward0>)

- attention is a communication mechanism, it gives a weighted sum of all nodes, with data-dependent weights
- no notion of space, it acts over a set of vectors. That's why we need positional embeddings. (differently than convolutions)
- each batch is independent
- in the encoder all tokens can communicate, in the decoder – only previous tokens. It also depends of the problem, e.g. for sentiment analysis we can use all and for translation usually only previous tokens.
- self-attention means keys, queries and values are from the same source,
in cross-attention queries are made from x and keys and values are from other source (e.g. encoder module).
- attention matrix can be scaled by 1/sqrt(d_k) d_k – head size. It keeps the variance of weights at 1. It's important for softmax.

In [19]:
# scaled attention
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) #* head_size**-0.5

print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)) # sharpenning to the highest value
k.var(), q.var(), wei.var()

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


(tensor(0.9213), tensor(0.9248), tensor(14.2450))